In [1]:
# import librart
import sklearn
import numpy
import pandas as pd
import pickle
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy
from scipy.sparse import csr_matrix
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import feature_select
import feture_process
from scipy.sparse import load_npz

In [25]:
# use countvectorizer
# use countvectorizer
# use countvectorizer
# use countvectorizer

# import data
train_df = pd.read_csv("./project_data_files/book_rating_train.csv")
test_df = pd.read_csv("./project_data_files/book_rating_test.csv")


In [26]:
# process Prublisher and Language
train_df["Language"], test_df["Language"] = feture_process.docclass_preprocess(train_df["Language"],test_df["Language"],10)
train_df["Publisher"], test_df["Publisher"] = feture_process.docclass_preprocess(train_df["Publisher"],test_df["Publisher"],200)

In [27]:
# use oneHotCode for Publisher (sklearn)
publisher_train_hot,publisher_test_hot = feture_process.process_OneHotEncoder(train_df,test_df,"Publisher")
train_df = pd.concat([train_df, publisher_train_hot], axis=1)
test_df = pd.concat([test_df, publisher_test_hot], axis=1)

In [28]:
# use oneHotCode for Language (sklearn)
language_train_hot,language_test_hot = feture_process.process_OneHotEncoder(train_df,test_df,"Language")
train_df = pd.concat([train_df, language_train_hot], axis=1)
test_df = pd.concat([test_df, language_test_hot], axis=1)


In [14]:
# use oneHotCode for Publisher (pandas)
publisher_train_hot,publisher_test_hot = feture_process.process_OneHotEncoder_pd(train_df,test_df,"Publisher")
train_df = pd.concat([train_df, publisher_train_hot], axis=1)
test_df = pd.concat([test_df, publisher_test_hot], axis=1)

In [15]:
# use oneHotCode for Language (pandas)
language_train_hot,language_test_hot = feture_process.process_OneHotEncoder_pd(train_df,test_df,"Language")
train_df = pd.concat([train_df, language_train_hot], axis=1)
test_df = pd.concat([test_df, language_test_hot], axis=1)

In [29]:
publisher_test_hot.shape,publisher_train_hot.shape

((5766, 5), (23063, 5))

In [30]:
# train_name_countvectorizer
train_name_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_name_countvectorizer.pkl", "rb"))
train_name_dic = train_name_countvectorizer.vocabulary_

# train_authors_countvectorizer
train_authors_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_authors_countvectorizer.pkl", "rb"))
train_authors_dic = train_authors_countvectorizer.vocabulary_

# train_desc_countvectorizer
train_desc_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_desc_countvectorizer.pkl", "rb"))
train_desc__dic = train_desc_countvectorizer.vocabulary_

# process vector features
train_name_features = train_name_countvectorizer.transform(train_df['Name'])
train_authors_features = train_authors_countvectorizer.transform(train_df['Authors'])
train_desc_features = train_desc_countvectorizer.transform(train_df['Description'])
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language', 'rating_label'])


In [31]:
# new sparse features
sparse_features = hstack([train_name_features, train_authors_features, train_desc_features])
# new train features
dense_features = csr_matrix(other_features_df.values)
train_features = hstack([sparse_features, dense_features])


# process test features
test_name_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_name_vec.npz')
test_authors_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_authors_vec.npz')
test_desc_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_desc_vec.npz')
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])

test_sparse_features = hstack([test_name_features, test_authors_features, test_desc_features])
new_dense_features = csr_matrix(test_other_features_df.values)
test_features = hstack([test_sparse_features, new_dense_features])

In [ ]:
# use countvectorizer
# use countvectorizer
# use countvectorizer
# use countvectorizer


# split train and testing dataset
X_train, X_val, y_train, y_val = train_test_split(train_features, train_df["rating_label"], test_size=0.2, random_state=42)
# random forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)



evaluate_pred = rf.predict(X_val)

# evaluate
accuracy = accuracy_score(y_val, evaluate_pred)
print('Validation Accuracy: ', accuracy)


y_pred = rf.predict(test_features)
output_df = pd.DataFrame({'rating_label': y_pred})
output_df.index += 1
output_df.index.name = 'id'
output_df.to_csv('./predictions.csv')


In [ ]:
# use doc2vec
# use doc2vec
# use doc2vec
# use doc2vec

# import data
train_df = pd.read_csv("./project_data_files/book_rating_train.csv")
test_df = pd.read_csv("./project_data_files/book_rating_test.csv")



# process vector features
train_name_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_name_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
train_authors_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_authors_doc2vec20.csv", index_col = False, delimiter = ',', header=None)
train_desc_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_desc_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language', 'rating_label'])
train_features = pd.concat([train_name_features, train_authors_features, train_desc_features, other_features_df], axis=1)


In [ ]:
# use doc2vec
# use doc2vec
# use doc2vec
# use doc2vec

# process test features
test_name_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_name_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
test_authors_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_authors_doc2vec20.csv", index_col = False, delimiter = ',', header=None)
test_desc_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_desc_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])
test_features = pd.concat([test_name_features, test_authors_features, test_desc_features, test_other_features_df], axis=1)
# split train and testing dataset
X_train, X_val, y_train, y_val = train_test_split(train_features, train_df["rating_label"], test_size=0.2, random_state=42)
# random forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)



evaluate_pred = rf.predict(X_val)

# evaluate
accuracy = accuracy_score(y_val, evaluate_pred)
print('Validation Accuracy: ', accuracy)


y_pred = rf.predict(test_features)
output_df = pd.DataFrame({'rating_label': y_pred})
output_df.index += 1
output_df.index.name = 'id'
output_df.to_csv('./predictions.csv')


In [ ]:
# mutualInformation selection
selected_features,selected_features_test = feature_select.MI(train_df,train_features,test_features,1000)

In [8]:
# use countvectorizer
# use countvectorizer
# use countvectorizer
# use countvectorizer



X_train, X_val, y_train, y_val = train_test_split(selected_features, train_df["rating_label"], test_size=0.2, random_state=66)
# random forest
rf = RandomForestClassifier(n_estimators=1500)
rf.fit(X_train, y_train)



evaluate_pred = rf.predict(X_val)

# evaluate
accuracy = accuracy_score(y_val, evaluate_pred)
print('Validation Accuracy: ', accuracy)

IndexError: index (127407) out of range

In [ ]:
type(selected_features_test)

In [ ]:
y_pred = rf.predict(selected_features_test)
output_df = pd.DataFrame({'rating_label': y_pred})
output_df.index += 1
output_df.index.name = 'id'
output_df.to_csv('./predictions.csv')

In [ ]:
# use countvectorizer
# use countvectorizer
# use countvectorizer
# use countvectorizer
# chi2 selection

selected_features,selected_features_test = feature_select.chi_square(train_df,train_features,test_features,1000)


X_train, X_val, y_train, y_val = train_test_split(selected_features, train_df["rating_label"], test_size=0.2, random_state=66)
# random forest
rf = RandomForestClassifier(n_estimators=1500)
rf.fit(X_train, y_train)



evaluate_pred = rf.predict(X_val)

# evaluate
accuracy = accuracy_score(y_val, evaluate_pred)
print('Validation Accuracy: ', accuracy)

In [ ]:
# export predction
y_pred = rf.predict(selected_features_test)
output_df = pd.DataFrame({'rating_label': y_pred})
output_df.index += 1
output_df.index.name = 'id'
output_df.to_csv('./predictions.csv')

In [ ]:
#import data after preprocessing
train_df = pd.read_csv("./project_data_files/book_rating_train.csv")
test_df = pd.read_csv("./project_data_files/book_rating_test.csv")
X_train = load_npz('x_train_combination_469.npz')
X_test = load_npz('x_test_469.npz')
y_train = train_df['rating_label']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=66)
# random forest
rf = RandomForestClassifier(n_estimators=1500)
rf.fit(X_train, y_train)
evaluate_pred = rf.predict(X_val)
# evaluate
accuracy = accuracy_score(y_val, evaluate_pred)
print('Validation Accuracy: ', accuracy)